# 训练VGG19模型，并保存权重

1 使用 VGG19

2 冻结训练分类器

3 finetune训练整个网络

### 1 import libs

In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.applications.vgg19 import preprocess_input

Using TensorFlow backend.


### 2 build pretrained model

In [8]:
input_tensor = Input((224, 224, 3))

base_model = VGG19(input_tensor=input_tensor, weights='imagenet', include_top=False)
base_model.summary()

m_out = base_model.output

flatten = Flatten(name='flatten')(m_out)

fc1 = Dense(4096, activation='relu', name='fc1')(flatten)
drop_fc1 = Dropout(0.5, name='drop_fc1')(fc1)

fc2 = Dense(4096, activation='relu', name='fc2')(drop_fc1)
drop_fc2 = Dropout(0.5, name='drop_fc2')(fc2)

predictions = Dense(4, activation='softmax', name='predictions')(drop_fc2)

for layer in base_model.layers:  
     layer.trainable = False

model = Model(inputs=base_model.input, outputs=predictions)
sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

### 3 build param

In [9]:
batch_size = 32
epochs = 1

total_imgs_num = 79309 + 3491
train_imgs_num = 79309
valid_imgs_num = 3491

img_size = (224, 224)

### 4 build data & train

In [10]:
img_gen_t = ImageDataGenerator(preprocessing_function = vgg19.preprocess_input)
train_generator = img_gen_t.flow_from_directory("../oct-dataset/OCT2017/train-ready/", img_size, shuffle=True, batch_size=batch_size)

img_gen_v = ImageDataGenerator(preprocessing_function = vgg19.preprocess_input)
valid_generator = img_gen_v.flow_from_directory("../oct-dataset/OCT2017/valid-ready/", img_size, shuffle=True, batch_size=batch_size)

model.fit_generator(generator=train_generator, steps_per_epoch=train_imgs_num//batch_size, epochs=epochs, verbose=1,
                   validation_data=valid_generator, validation_steps=valid_imgs_num//batch_size)

model.save_weights("vgg19_first_train.h5")

Found 79309 images belonging to 4 classes.
Found 4175 images belonging to 4 classes.
Epoch 1/1
2479/2478 [==============================] - 362s 146ms/step - loss: 8.9416 - acc: 0.4450 - val_loss: 8.7366 - val_acc: 0.4580


### 5 Finetune

In [5]:
input_tensor = Input((224, 224, 3))

base_model = VGG19(input_tensor=input_tensor, weights='imagenet', include_top=False)

m_out = base_model.output

flatten = Flatten(name='flatten')(m_out)

fc1 = Dense(4096, activation='relu', name='fc1')(flatten)
drop_fc1 = Dropout(0.5, name='drop_fc1')(fc1)

fc2 = Dense(4096, activation='relu', name='fc2')(drop_fc1)
drop_fc2 = Dropout(0.5, name='drop_fc2')(fc2)

predictions = Dense(4, activation='softmax', name='predictions')(drop_fc2)

model = Model(inputs=base_model.input, outputs=predictions)
model.load_weights('vgg19_first_train.h5')
sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [6]:
batch_size = 32
epochs = 10

total_imgs_num = 79309 + 3491
train_imgs_num = 79309
valid_imgs_num = 3491

img_size = (224, 224)

img_gen_t = ImageDataGenerator(preprocessing_function = vgg19.preprocess_input)
train_generator = img_gen_t.flow_from_directory("../oct-dataset/OCT2017/train-ready/", img_size, shuffle=True, batch_size=batch_size)

img_gen_v = ImageDataGenerator(preprocessing_function = vgg19.preprocess_input)
valid_generator = img_gen_v.flow_from_directory("../oct-dataset/OCT2017/valid-ready/", img_size, shuffle=True, batch_size=batch_size)


model.fit_generator(generator=train_generator, steps_per_epoch=train_imgs_num//batch_size, epochs=epochs, verbose=1,
                   validation_data=valid_generator, validation_steps=valid_imgs_num//batch_size)

model.save_weights("vgg19_finetune.h5")

Found 79309 images belonging to 4 classes.
Found 4175 images belonging to 4 classes.
Epoch 1/10
2479/2478 [==============================] - 930s 375ms/step - loss: 8.9463 - acc: 0.4450 - val_loss: 8.7366 - val_acc: 0.4580
Epoch 2/10
1507/2478 [=================>............] - ETA: 5:57 - loss: 8.9366 - acc: 0.4456

KeyboardInterrupt: 

In [ ]:
batch_size = 24
epochs = 1

total_imgs_num = 100 + 1000
train_imgs_num = 100
valid_imgs_num = 1000

img_size = (299, 299)

img_gen_t = ImageDataGenerator()
train_generator = img_gen_t.flow_from_directory("../oct-dataset/OCT2017/train-ready/", img_size, shuffle=True, batch_size=batch_size)

img_gen_v = ImageDataGenerator()
valid_generator = img_gen_v.flow_from_directory("../oct-dataset/OCT2017/test-ready/", img_size, shuffle=True, batch_size=batch_size)


model.fit_generator(generator=train_generator, steps_per_epoch=train_imgs_num//batch_size, epochs=epochs, verbose=1,
                   validation_data=valid_generator, validation_steps=valid_imgs_num//batch_size)

### 5 load weights & predict

In [43]:
input_tensor = Input((299, 299, 3))
x = Lambda(xception.preprocess_input)(input_tensor)

base_model = Xception(input_tensor=x, weights=None, include_top=False)
m_out = base_model.output
p_out = GlobalAveragePooling2D()(m_out)
p_out = Dropout(1.0)(p_out)
fc_out = Dense(1024, activation='relu')(p_out)
fc_out = Dropout(1.0)(fc_out)
predictions = Dense(3, activation='softmax')(fc_out)
model = Model(inputs=base_model.input, outputs=predictions)
model.load_weights('Xception.h5')

In [44]:
img_gen_s = ImageDataGenerator()
submit_generator = img_gen_t.flow_from_directory("../dataset/test", img_size, shuffle=False, batch_size=2)

predictions = []
for i, batch_data in enumerate(submit_generator):
    if (i > 1):
        break
    batch_img_data = np.array(batch_data[0])
    predictions.extend(model.predict_on_batch(batch_img_data))
    print(i)
    
print(predictions)

Found 14 images belonging to 3 classes.
0
1
[array([ 0.30176267,  0.35524741,  0.34298989], dtype=float32), array([ 0.28475785,  0.34978592,  0.36545616], dtype=float32), array([ 0.3018229 ,  0.35525236,  0.34292471], dtype=float32), array([ 0.29986629,  0.35433695,  0.34579682], dtype=float32)]


In [45]:
print(np.array(predictions))

[[ 0.30176267  0.35524741  0.34298989]
 [ 0.28475785  0.34978592  0.36545616]
 [ 0.3018229   0.35525236  0.34292471]
 [ 0.29986629  0.35433695  0.34579682]]


### 6 write predictions 2 CSV